# Excercise 8 Model-based Monte Carlo tree search on Santorini environment

## Goals:
- able to adapt an environment to work with MCTS
- implement MCTS

## Prerequisites

- pull updates of https://github.com/phizaz/chula_rl 
- you will need to write code in python (not just in the notebooks)
- a good code editor is preferable

## Things you need to implement:

1. **MCTS** `chula_rl.alphazero.mcts`, Monte-Carlo tree search file. We have coded the template, but you need to fill in the blanks.
2. **Game interface** `chula_rl.alphazero.santorini.game`, You need to complete the interface between an environment and MCTS. We have coded the template, you need to fill the blanks.
3. **Network** `chula_rl.alphazero.santorini.net.tf`, the main neural network shoud be implemented in TF2. 

## Hints

`chula_rl.alphazero.othello` is an example of adapting an environment (the game of Othello) to work with MCTS interface. You can take it as an inspiration for the Santorini. 

However, you still need to implement the MCTS code which is not provided. 



Disclaimer: most of the code in the repo is not developed by us. It is from an annonymous origin for now. 

## Step 0 Understanding the game of Santorini

Youtube explanation from the author of the game: https://www.youtube.com/watch?v=EzHAykZTCHU

We don't really play the "original" Santorini, we play a simpler one in which there are no "god powers" cards.

Python implementation of the game is available at: https://github.com/cstorm125/santorini (hats off to Charin 🙏)

We have made some changes to it ... so use the one in `chula_rl.alphazero.santorini.santorinigo.environment` instead. 

Details are seen in the repo under sections "Setup" and "Each turn".


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

## Step 1 implement the game interface 

You might want to play around to make sure that it does what it should do. Checking each method one by one is a good idea.

In [3]:
from chula_rl.alphazero.santorini.game import SantoriniGame

game = SantoriniGame()

In [4]:
board = game.getInitBoard()
board

array([[[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0]],

       [[ 0,  0, -1,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  2],
        [ 0,  0,  0,  0,  0],
        [ 0,  0, -2,  0,  0]],

       [[ 0,  0,  0,  0,  0],
        [ 0, 22,  0,  0,  0],
        [ 0,  0, 18,  0,  0],
        [ 0,  0,  0, 14,  0],
        [ 0,  0,  0,  0, 18]]], dtype=int64)

In [5]:
board_size = game.getBoardSize()
board_size

(5, 5)

In [6]:
game.getActionSize()

128

In [7]:
game.getNextState(board,-1,1)

(array([[[ 0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0]],
 
        [[ 0,  0, -1,  0,  0],
         [ 0,  0,  0,  0,  0],
         [ 1,  0,  0,  0,  2],
         [ 0,  0,  0,  0,  0],
         [ 0,  0, -2,  0,  0]],
 
        [[ 0,  0,  0,  0,  0],
         [ 0, 22,  0,  0,  0],
         [ 0,  0, 18,  0,  0],
         [ 0,  0,  0, 14,  0],
         [ 0,  0,  0,  0, 18]]], dtype=int64), 1)

In [8]:
game.getValidMoves(board,-1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
game.getGameEnded(board,-1)

0

In [10]:
game.getCanonicalForm(board,-1)

array([[[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0]],

       [[ 0,  0,  1,  0,  0],
        [ 0,  0,  0,  0,  0],
        [-1,  0,  0,  0, -2],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  2,  0,  0]],

       [[ 0,  0,  0,  0,  0],
        [ 0, 22,  0,  0,  0],
        [ 0,  0, 18,  0,  0],
        [ 0,  0,  0, 14,  0],
        [ 0,  0,  0,  0, 18]]], dtype=int64)

In [11]:
pi = []
for i in range(5):
    pi.append([])
    for j in range(5):
        pi[i].append(0)
    pi[i] = np.array(pi[i])
pi = np.array(pi)
game.getSymmetries(board,pi)

[(array([[[ 0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0]],
  
         [[ 0,  0, -1,  0,  0],
          [ 0,  0,  0,  0,  0],
          [ 1,  0,  0,  0,  2],
          [ 0,  0,  0,  0,  0],
          [ 0,  0, -2,  0,  0]],
  
         [[ 0,  0,  0,  0,  0],
          [ 0, 22,  0,  0,  0],
          [ 0,  0, 18,  0,  0],
          [ 0,  0,  0, 14,  0],
          [ 0,  0,  0,  0, 18]]], dtype=int64), array([[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]))]

In [12]:
display = game.stringRepresentation(board)

## Step 2 implement the neural network

Neural network in this case hase "two" heads.

One for action distribution. One for state-value prediction.

Make sure that the output shape makes sense.

**Feel free to change the existing code.**

In [13]:
from chula_rl.alphazero.santorini.net.tf import Backbone, SantoriniNet

In [14]:
config = {
    "n_ch":5,
    "lr":0.01,
    "dropout":0.1
}
backbone = Backbone(game,config)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
santo = SantoriniNet(game,config)

In [16]:
board.shape

(3, 5, 5)

In [17]:
(pi,v) = santo.predict(board)

In [18]:
pi.shape,v.shape

((128,), (1,))

## Step 3 implement MCTS

You need to implement the file `chula_rl.alphazero.mcts`. We have already provided some structure of it. The code should be fairly close to the pseucode in the classroom. 

In [19]:
from chula_rl.alphazero.mcts import MCTS

In [20]:
mcts = MCTS(game,santo,1,0)

In [21]:
mcts.search(board)

1

## Step 4 Train an agent with MCTS

To train an agent under the MCTS framework there are a few ingredients to consider (all of which are provided). 

We have provided:

- **Coach** `chula_rl.alphazero.coach` this is the main "traning loop". Basically, you call `coach.learn`
- **Arena** which is used by **Coach** for competing two policies to see which one fares better. **Coach** uses this to determine if the improved policy is really better, if not it is discarded.

Coach will automatically save neural nets for each step of progresses. You can control where to save in `Args` (default `./checkpoint/`)

### Advice

Training takes a very longggg time. If you want to debug, set args so that it would run fast to make sure your code work. Only when you are pretty sure about your code, run with a larger setting! 👌

In [22]:
from chula_rl.alphazero.coach import *

# config for the coach
args = Args() 
config = {
    "n_ch":5,
    "lr":0.01,
    "dropout":0.1,
    "n_bs":100,
    "n_ep":10
}

def make_net():
    game = SantoriniGame()
    santo = SantoriniNet(game,config)
    return santo

g = SantoriniGame()

c = Coach(g, make_net, args)
c.learn()

------ITER 1------
Self Play |################################| (10/10) Eps Time: 0.275s | Total: 0:00:02 | ETA: 0:00:01
Checkpoint Directory exists! 
(238, 25)
Epoch 1/10
238/238 [==============================] - ETA: 0s - loss: 5.8538 - pi_loss: 4.8374 - v_loss: 1.01 - 0s 561us/sample - loss: 6.3279 - pi_loss: 4.9914 - v_loss: 1.3058
Epoch 2/10
238/238 [==============================] - ETA: 0s - loss: 5.9345 - pi_loss: 4.9052 - v_loss: 1.02 - 0s 101us/sample - loss: 5.9471 - pi_loss: 4.8843 - v_loss: 1.0872
Epoch 3/10
238/238 [==============================] - ETA: 0s - loss: 5.4405 - pi_loss: 4.7469 - v_loss: 0.69 - 0s 88us/sample - loss: 5.4362 - pi_loss: 4.7304 - v_loss: 0.6328
Epoch 4/10
238/238 [==============================] - ETA: 0s - loss: 5.1739 - pi_loss: 4.6974 - v_loss: 0.47 - 0s 88us/sample - loss: 5.1406 - pi_loss: 4.6918 - v_loss: 0.4683
Epoch 5/10
238/238 [==============================] - ETA: 0s - loss: 5.1022 - pi_loss: 4.6832 - v_loss: 0.41 - 0s 92us/sample - 

850/850 [==============================] - ETA: 0s - loss: 4.6816 - pi_loss: 4.5268 - v_loss: 0.15 - 0s 69us/sample - loss: 4.6000 - pi_loss: 4.5042 - v_loss: 0.0907
Epoch 9/10
850/850 [==============================] - ETA: 0s - loss: 4.4777 - pi_loss: 4.4343 - v_loss: 0.04 - 0s 70us/sample - loss: 4.5130 - pi_loss: 4.4694 - v_loss: 0.0408
Epoch 10/10
850/850 [==============================] - ETA: 0s - loss: 4.4471 - pi_loss: 4.4180 - v_loss: 0.02 - 0s 72us/sample - loss: 4.4688 - pi_loss: 4.4403 - v_loss: 0.0281
PITTING AGAINST PREVIOUS VERSION
Arena.playGames |################################| (40/40) Eps Time: 0.380s | Total: 0:00:15 | ETA: 0:00:01 | one/two: 20/20
NEW/PREV WINS : 20 / 20 ; DRAWS : 0
REJECTING NEW MODEL
------ITER 5------
Self Play |################################| (10/10) Eps Time: 0.256s | Total: 0:00:02 | ETA: 0:00:01
Checkpoint Directory exists! 
(1081, 25)
Epoch 1/10
1081/1081 [==============================] - ETA: 0s - loss: 5.8258 - pi_loss: 4.8377 - v_lo

1493/1493 [==============================] - ETA: 0s - loss: 4.5858 - pi_loss: 4.5024 - v_loss: 0.08 - ETA: 0s - loss: 4.5550 - pi_loss: 4.4895 - v_loss: 0.06 - 0s 73us/sample - loss: 4.5502 - pi_loss: 4.4856 - v_loss: 0.0644
Epoch 9/10
1493/1493 [==============================] - ETA: 0s - loss: 4.5109 - pi_loss: 4.4523 - v_loss: 0.05 - ETA: 0s - loss: 4.4974 - pi_loss: 4.4523 - v_loss: 0.04 - 0s 79us/sample - loss: 4.5021 - pi_loss: 4.4588 - v_loss: 0.0435
Epoch 10/10
1493/1493 [==============================] - ETA: 0s - loss: 4.4012 - pi_loss: 4.3865 - v_loss: 0.01 - ETA: 0s - loss: 4.4633 - pi_loss: 4.4224 - v_loss: 0.04 - 0s 69us/sample - loss: 4.4716 - pi_loss: 4.4345 - v_loss: 0.0371
PITTING AGAINST PREVIOUS VERSION
Arena.playGames |################################| (40/40) Eps Time: 0.245s | Total: 0:00:09 | ETA: 0:00:01 | one/two: 20/20
NEW/PREV WINS : 20 / 20 ; DRAWS : 0
REJECTING NEW MODEL
------ITER 8------
Self Play |################################| (10/10) Eps Time: 0.2

2215/2215 [==============================] - ETA: 0s - loss: 4.9205 - pi_loss: 4.6568 - v_loss: 0.26 - ETA: 0s - loss: 4.9986 - pi_loss: 4.6394 - v_loss: 0.35 - ETA: 0s - loss: 4.9900 - pi_loss: 4.6290 - v_loss: 0.36 - 0s 63us/sample - loss: 4.9877 - pi_loss: 4.6319 - v_loss: 0.3578
Epoch 4/10
2215/2215 [==============================] - ETA: 0s - loss: 4.9035 - pi_loss: 4.6154 - v_loss: 0.28 - ETA: 0s - loss: 4.8742 - pi_loss: 4.6026 - v_loss: 0.27 - ETA: 0s - loss: 4.8794 - pi_loss: 4.6018 - v_loss: 0.27 - 0s 63us/sample - loss: 4.8837 - pi_loss: 4.6042 - v_loss: 0.2839
Epoch 5/10
2215/2215 [==============================] - ETA: 0s - loss: 4.8751 - pi_loss: 4.6001 - v_loss: 0.27 - ETA: 0s - loss: 4.8190 - pi_loss: 4.5849 - v_loss: 0.23 - ETA: 0s - loss: 4.8087 - pi_loss: 4.5849 - v_loss: 0.22 - 0s 71us/sample - loss: 4.8074 - pi_loss: 4.5876 - v_loss: 0.2326
Epoch 6/10
2215/2215 [==============================] - ETA: 0s - loss: 4.7726 - pi_loss: 4.5843 - v_loss: 0.18 - ETA: 0s - lo

2591/2591 [==============================] - ETA: 0s - loss: 4.6629 - pi_loss: 4.5232 - v_loss: 0.13 - ETA: 0s - loss: 4.6622 - pi_loss: 4.5336 - v_loss: 0.12 - ETA: 0s - loss: 4.6634 - pi_loss: 4.5351 - v_loss: 0.12 - ETA: 0s - loss: 4.6760 - pi_loss: 4.5393 - v_loss: 0.13 - 0s 70us/sample - loss: 4.6745 - pi_loss: 4.5382 - v_loss: 0.1361
Epoch 8/10
2591/2591 [==============================] - ETA: 0s - loss: 4.5444 - pi_loss: 4.5017 - v_loss: 0.04 - ETA: 0s - loss: 4.6065 - pi_loss: 4.5095 - v_loss: 0.09 - ETA: 0s - loss: 4.6143 - pi_loss: 4.5165 - v_loss: 0.09 - 0s 63us/sample - loss: 4.6231 - pi_loss: 4.5179 - v_loss: 0.1053
Epoch 9/10
2591/2591 [==============================] - ETA: 0s - loss: 4.5397 - pi_loss: 4.4852 - v_loss: 0.05 - ETA: 0s - loss: 4.5701 - pi_loss: 4.4857 - v_loss: 0.08 - ETA: 0s - loss: 4.5887 - pi_loss: 4.4973 - v_loss: 0.09 - 0s 74us/sample - loss: 4.5880 - pi_loss: 4.4975 - v_loss: 0.0906
Epoch 10/10
2591/2591 [==============================] - ETA: 0s - l

...

4233/4233 [==============================] - ETA: 0s - loss: 4.8599 - pi_loss: 4.5962 - v_loss: 0.26 - ETA: 0s - loss: 4.7884 - pi_loss: 4.5803 - v_loss: 0.20 - ETA: 0s - loss: 4.7891 - pi_loss: 4.5810 - v_loss: 0.20 - ETA: 0s - loss: 4.7870 - pi_loss: 4.5845 - v_loss: 0.20 - ETA: 0s - loss: 4.7908 - pi_loss: 4.5874 - v_loss: 0.20 - 0s 65us/sample - loss: 4.7862 - pi_loss: 4.5876 - v_loss: 0.1965
Epoch 7/10
4233/4233 [==============================] - ETA: 0s - loss: 4.7210 - pi_loss: 4.5649 - v_loss: 0.15 - ETA: 0s - loss: 4.7236 - pi_loss: 4.5678 - v_loss: 0.15 - ETA: 0s - loss: 4.7180 - pi_loss: 4.5747 - v_loss: 0.14 - ETA: 0s - loss: 4.7293 - pi_loss: 4.5729 - v_loss: 0.15 - ETA: 0s - loss: 4.7268 - pi_loss: 4.5730 - v_loss: 0.15 - ETA: 0s - loss: 4.7351 - pi_loss: 4.5743 - v_loss: 0.16 - 0s 71us/sample - loss: 4.7298 - pi_loss: 4.5727 - v_loss: 0.1567
Epoch 8/10
4233/4233 [==============================] - ETA: 0s - loss: 4.7082 - pi_loss: 4.5726 - v_loss: 0.13 - ETA: 0s - loss: 4

Epoch 3/10
4170/4170 [==============================] - ETA: 0s - loss: 5.0019 - pi_loss: 4.6347 - v_loss: 0.36 - ETA: 0s - loss: 5.0114 - pi_loss: 4.6301 - v_loss: 0.38 - ETA: 0s - loss: 5.0145 - pi_loss: 4.6258 - v_loss: 0.38 - ETA: 0s - loss: 5.0100 - pi_loss: 4.6280 - v_loss: 0.38 - ETA: 0s - loss: 5.0179 - pi_loss: 4.6305 - v_loss: 0.38 - 0s 66us/sample - loss: 5.0192 - pi_loss: 4.6335 - v_loss: 0.3852
Epoch 4/10
4170/4170 [==============================] - ETA: 0s - loss: 4.9727 - pi_loss: 4.6436 - v_loss: 0.32 - ETA: 0s - loss: 4.8747 - pi_loss: 4.6210 - v_loss: 0.25 - ETA: 0s - loss: 4.8936 - pi_loss: 4.6180 - v_loss: 0.27 - ETA: 0s - loss: 4.8924 - pi_loss: 4.6181 - v_loss: 0.27 - ETA: 0s - loss: 4.8962 - pi_loss: 4.6150 - v_loss: 0.28 - 0s 66us/sample - loss: 4.9065 - pi_loss: 4.6154 - v_loss: 0.2902
Epoch 5/10
4170/4170 [==============================] - ETA: 0s - loss: 4.8276 - pi_loss: 4.5812 - v_loss: 0.24 - ETA: 0s - loss: 4.8142 - pi_loss: 4.5929 - v_loss: 0.22 - ETA: 0

4217/4217 [==============================] - ETA: 0s - loss: 4.5337 - pi_loss: 4.5110 - v_loss: 0.02 - ETA: 0s - loss: 4.6162 - pi_loss: 4.5290 - v_loss: 0.08 - ETA: 0s - loss: 4.6220 - pi_loss: 4.5275 - v_loss: 0.09 - ETA: 0s - loss: 4.6276 - pi_loss: 4.5311 - v_loss: 0.09 - ETA: 0s - loss: 4.6352 - pi_loss: 4.5345 - v_loss: 0.10 - ETA: 0s - loss: 4.6413 - pi_loss: 4.5355 - v_loss: 0.10 - 0s 70us/sample - loss: 4.6423 - pi_loss: 4.5354 - v_loss: 0.1053
PITTING AGAINST PREVIOUS VERSION
Arena.playGames |################################| (40/40) Eps Time: 0.239s | Total: 0:00:09 | ETA: 0:00:01 | one/two: 20/20
NEW/PREV WINS : 20 / 20 ; DRAWS : 0
REJECTING NEW MODEL
------ITER 97------
Self Play |################################| (10/10) Eps Time: 0.246s | Total: 0:00:02 | ETA: 0:00:01
len(trainExamplesHistory) = 21  => remove the oldest trainExamples
Checkpoint Directory exists! 
(4248, 25)
Epoch 1/10
4248/4248 [==============================] - ETA: 0s - loss: 5.9112 - pi_loss: 4.8418 -

4283/4283 [==============================] - ETA: 0s - loss: 4.7387 - pi_loss: 4.5624 - v_loss: 0.17 - ETA: 0s - loss: 4.7705 - pi_loss: 4.5932 - v_loss: 0.17 - ETA: 0s - loss: 4.8108 - pi_loss: 4.5952 - v_loss: 0.21 - ETA: 0s - loss: 4.8292 - pi_loss: 4.6040 - v_loss: 0.22 - ETA: 0s - loss: 4.8407 - pi_loss: 4.6066 - v_loss: 0.23 - 0s 62us/sample - loss: 4.8378 - pi_loss: 4.6065 - v_loss: 0.2314
Epoch 7/10
4283/4283 [==============================] - ETA: 0s - loss: 4.7754 - pi_loss: 4.5870 - v_loss: 0.18 - ETA: 0s - loss: 4.7701 - pi_loss: 4.5897 - v_loss: 0.18 - ETA: 0s - loss: 4.7522 - pi_loss: 4.5891 - v_loss: 0.16 - ETA: 0s - loss: 4.7602 - pi_loss: 4.5902 - v_loss: 0.17 - ETA: 0s - loss: 4.7760 - pi_loss: 4.5916 - v_loss: 0.18 - 0s 67us/sample - loss: 4.7707 - pi_loss: 4.5899 - v_loss: 0.1804
Epoch 8/10
4283/4283 [==============================] - ETA: 0s - loss: 4.6544 - pi_loss: 4.5417 - v_loss: 0.11 - ETA: 0s - loss: 4.7035 - pi_loss: 4.5634 - v_loss: 0.14 - ETA: 0s - loss: 4

4352/4352 [==============================] - ETA: 0s - loss: 5.0834 - pi_loss: 4.6456 - v_loss: 0.43 - ETA: 0s - loss: 5.0551 - pi_loss: 4.6497 - v_loss: 0.40 - ETA: 0s - loss: 5.0687 - pi_loss: 4.6464 - v_loss: 0.42 - ETA: 0s - loss: 5.0748 - pi_loss: 4.6452 - v_loss: 0.42 - ETA: 0s - loss: 5.0747 - pi_loss: 4.6409 - v_loss: 0.43 - ETA: 0s - loss: 5.0682 - pi_loss: 4.6396 - v_loss: 0.42 - 0s 69us/sample - loss: 5.0662 - pi_loss: 4.6402 - v_loss: 0.4242
Epoch 4/10
4352/4352 [==============================] - ETA: 0s - loss: 4.8990 - pi_loss: 4.6299 - v_loss: 0.26 - ETA: 0s - loss: 4.9974 - pi_loss: 4.6346 - v_loss: 0.36 - ETA: 0s - loss: 4.9963 - pi_loss: 4.6338 - v_loss: 0.36 - ETA: 0s - loss: 5.0018 - pi_loss: 4.6306 - v_loss: 0.37 - ETA: 0s - loss: 4.9794 - pi_loss: 4.6250 - v_loss: 0.35 - ETA: 0s - loss: 4.9742 - pi_loss: 4.6246 - v_loss: 0.34 - 0s 66us/sample - loss: 4.9718 - pi_loss: 4.6244 - v_loss: 0.3452
Epoch 5/10
4352/4352 [==============================] - ETA: 0s - loss: 4

## Step 5 comparing agents

You could use **Arena** to compare many kinds of agents. You could compete the best agent in the `./checkpoint/` to compare with other versions of itself. Or with a random agent. 

Note: a baseline from us might be released later on.

### Provided agents in `chula_rl.alphazero.santorini.players`

- Human: you can control an agent by yourself, albeit you need to know the exact command for it (which means you need to dig deeper into the codebase 😉)
- Random: a random player. Your policy should be better than random agent at the very least. Show us!

In [200]:
from chula_rl.alphazero.arena import Arena

In [201]:
from chula_rl.alphazero.santorini.players import RandomPlayer,HumanPlayer,AgentPlayer

In [202]:
model = make_net()

In [203]:
model.load_checkpoint(folder='checkpoint',
                                      filename='temp.pth.tar')

In [204]:
game = SantoriniGame()
randomplayer = RandomPlayer(game)
agentplayer = AgentPlayer(game,model)

In [205]:
agentplayer(board)

112

In [206]:
arena = Arena(randomplayer,agentplayer,game)

In [207]:
# -1 = second player win
arena.playGame()

-1

## How do we grade?

We look for the effort you put into making MCTS work with Santorini. 